In [1]:
import onnx
# ommx_model_pth = "/home/bydguikong/yy_ws/PlanScope/onnx/simplified_model.onnx"
ommx_model_pth = "/home/bydguikong/yy_ws/PlanScope/onnx/model.onnx"

model = onnx.load(ommx_model_pth)
inferred_model = onnx.shape_inference.infer_shapes(model)
onnx.checker.check_model(inferred_model)

AttributeError: module 'onnx' has no attribute 'load'

In [ ]:
import onnxruntime as ort
import numpy as np

# 加载ONNX模型（请确保模型文件路径正确）
print("开始加载")
# session = ort.InferenceSession("/home/bydguikong/yy_ws/PlanScope/onnx/model.onnx")
so = ort.SessionOptions()
# so.log_severity_level = 0
so.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL  # 或者尝试其他级别

In [ ]:
session = ort.InferenceSession(ommx_model_pth, sess_options=so,
                               providers=['CPUExecutionProvider'])
print("加载成功")


In [ ]:
import torch
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
# 获取模型的输入名称和形状
input_names = [
    "agent.position", "agent.heading", "agent.velocity", "agent.shape",
    "agent.category", "agent.valid_mask", #"agent.target",
    "map.point_position", "map.point_vector", "map.point_orientation", #"map.point_side",
    "map.polygon_center", #"map.polygon_position", #"map.polygon_orientation",
    "map.polygon_type", "map.polygon_on_route", "map.polygon_tl_status",
    "map.polygon_has_speed_limit", "map.polygon_speed_limit", #"map.polygon_road_block_id",
    "map.valid_mask",
    "reference_line.position", "reference_line.vector", "reference_line.orientation",
    "reference_line.valid_mask", #"reference_line.future_projection",
    "static_objects.position", "static_objects.heading", "static_objects.shape",
    "static_objects.category", "static_objects.valid_mask",
    "current_state", #"origin", "angle"
]

# for inp in session.get_inputs():
#     print(f"Input name: {inp.name}, Input shape:{inp.shape}")

agent_num = 15
map_line_num = 200
object_num = 100
# 构造一个示例输入数据（这里使用随机数据，实际使用时替换为真实数据）
input_data = [
    # agent 部分
    torch.randn(1, agent_num, 21, 2, dtype= torch.float32, device=device),            # 0: agent.position
    torch.randn(1, agent_num, 21, dtype= torch.float32, device=device),                # 1: agent.heading
    torch.randn(1, agent_num, 21, 2, dtype= torch.float32, device=device),             # 2: agent.velocity
    torch.randn(1, agent_num, 21, 2, dtype= torch.float32, device=device),             # 3: agent.shape
    torch.randint(0, 2, (1, agent_num), dtype=torch.int8, device=device),          # 4: agent.category
    torch.randint(0, 2, (1, agent_num, 21), dtype=torch.bool, device=device),  # 5: agent.valid_mask
    # torch.randn(1, agent_num, 0, 3, dtype= torch.float32, device=device),              # 6: agent.target

    # map 部分
    torch.randn(1, map_line_num, 3, 20, 2, dtype= torch.float32, device=device),         # 7: map.point_position
    torch.randn(1, map_line_num, 3, 20, 2, dtype= torch.float32, device=device),         # 8: map.point_vector
    torch.randn(1, map_line_num, 3, 20, dtype= torch.float32, device=device),            # 9: map.point_orientation
    # torch.randn(1, map_line_num, 3, dtype= torch.float32, device=device),                # 10: map.point_side
    torch.randn(1, map_line_num, 3, dtype= torch.float32, device=device),                # 11: map.polygon_center
    # torch.randn(1, map_line_num, 2, dtype= torch.float32, device=device),                # 12: map.polygon_position
    # torch.randn(1, map_line_num, dtype= torch.float32, device=device),                   # 13: map.polygon_orientation
    torch.randint(0, 3, (1, map_line_num), dtype=torch.int8, device=device),         # 14: map.polygon_type
    torch.randint(0, 2, (1, map_line_num), dtype=torch.bool, device=device),  # 15: map.polygon_on_route
    torch.randint(0, 1, (1, map_line_num), dtype=torch.int8, device=device),  # 16: map.polygon_tl_status
    torch.randint(0, 2, (1, map_line_num), dtype=torch.bool, device=device),  # 17: map.polygon_has_speed_limit
    torch.randn(1, map_line_num, dtype= torch.float32, device=device),                   # 18: map.polygon_speed_limit
    # torch.randint(0, 1000, (1, map_line_num), dtype=torch.int32, device=device),  # 19: map.polygon_road_block_id
    torch.randint(0, 2, (1, map_line_num, 20), dtype=torch.bool, device=device),  # 20: map.valid_mask

    # reference_line 部分
    torch.randn(1, 1, 120, 2, dtype= torch.float32, device=device),             # 21: reference_line.position
    torch.randn(1, 1, 120, 2, dtype= torch.float32, device=device),             # 22: reference_line.vector
    torch.randn(1, 1, 120, dtype= torch.float32, device=device),                # 23: reference_line.orientation
    torch.randint(0, 2, (1, 1, 120), dtype=torch.bool, device=device), # 24: reference_line.valid_mask
    # torch.randn(1, 1, 8, 2, dtype= torch.float32, device=device),               # 25: reference_line.future_projection

    # static_objects 部分
    torch.randn(1, object_num, 2, dtype= torch.float32, device=device),                 # 26: static_objects.position
    torch.randn(1, object_num, dtype= torch.float32, device=device),                    # 27: static_objects.heading
    torch.randn(1, object_num, 2, dtype= torch.float32, device=device),                 # 28: static_objects.shape
    torch.randint(1, 4, (1, object_num), dtype= torch.float32, device=device),          # 29: static_objects.category
    torch.randint(0, 2, (1, object_num), dtype=torch.bool, device=device),  # 30: static_objects.valid_mask

    # 其他部分
    torch.randn(1, 7, dtype= torch.float32, device=device),                     # 31: current_state
    # torch.randn(1, 2, dtype= torch.float32, device=device),                     # 32: origin
    # torch.randn(1, dtype= torch.float32, device=device),                        # 33: angle
]
inputs = {name: tensor.cpu().numpy() for name, tensor in zip(input_names, input_data)}
# for k,v in inputs.items():
#     print(f"Input name: {k}, Input shape: {v.shape}")
import time
print("开始推理")
import plotly.graph_objects as go
from plotly.colors import qualitative
for i in range(10):
    start_time = time.time()
    # 执行推理
    outputs = session.run(None, inputs)
    end_time = time.time()
    # print("推理结果:", outputs)
    # print(f"output.type = {type(outputs)}")
    # print(f"output.length = {len(outputs)}")
    # print(f"output[0].type = {type(outputs[0])}")
    # output_names=[
    #     "T_trajectory",           # 多模态轨迹（x,y,cos,sin,vx,vy）
    #     "S_probability",          # 多模态轨迹打分
    #     "P_prediction",           # Agent运动预测（x,y,cos,sin,vx,vy）  
    #     "output_prediction",      # Agent运动预测（x,y,yaw,vx,vy） 
    #     "output_trajectory",      # 多模态轨迹（x,y,yaw）
    #     "candidate_trajectories"  # 打分最高轨迹（x,y,yaw）
    # ]# 
    # i = 0
    # for item in outputs:
    #     print(f"{output_names[i]}.shape = {item.shape}")
    #     print(f"{output_names[i]}.dtype = {item.dtype}")
    #     i = i+1
    
    print("运算耗时: {:.3f} 秒".format(end_time - start_time))
    output_trajectory = outputs[4].squeeze()
    # print("output_trajectory shape:", output_trajectory.shape)
    # output_trajectory 的形状为 (traj_len, 3) x,y,yaw 绘制x,y 
    
    # 提取 x,y 坐标用于绘制轨迹
    traj = output_trajectory[:, :2]
    # print(f"x:{traj[:,0]}")
    # print(f"y:{traj[:,1]}")
    # 使用 Plotly 绘制轨迹图，x,y等比例显示
    fig_traj = go.Figure()
    fig_traj.add_trace(go.Scatter(
        x=traj[:, 0],
        y=traj[:, 1],
        mode='lines+markers',
        name='Output Trajectory'
    ))
    fig_traj.update_layout(
        title="Output Trajectory",
        xaxis_title="X",
        yaxis_title="Y",
        template="plotly_white",
        yaxis=dict(scaleanchor="x", scaleratio=1)
    )
    fig_traj.show(renderer="notebook")